In [ ]:
import os
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from glob import glob
from tslearn import barycenters
from scipy.stats import zscore

In [2]:
# Define path for derivatives directory
deriv_dir = "/Users/abry4213/data/Cogitate_MEG/derivatives"
MEG_time_series_dir = f"{deriv_dir}/MEG_time_series"

# Initialize list for pyspi results
all_barycenter_res_list = []

barycenter_method_dict = {'euclidean': barycenters.euclidean_barycenter,
                          'softdtw': barycenters.softdtw_barycenter,
                          'dtw': barycenters.dtw_barycenter_averaging,
                          'sgddtw': barycenters.dtw_barycenter_averaging_subgradient}

In [5]:
all_barycenter_res_list = []

# Load in individual subjects' MEG time series
for subject_averaged_TS_file in glob(f"{MEG_time_series_dir}/*_ses-1_meg_1000ms_all_time_series.csv")[0:1]:
    subject_ID = os.path.basename(subject_averaged_TS_file).replace("_ses-1_meg_1000ms_all_time_series.csv", "")
    subject_averaged_TS = pd.read_csv(subject_averaged_TS_file)

    # Iterate over each stimulus_type and relevance_type
    for this_stim in subject_averaged_TS.stimulus_type.unique():
        for this_rel in subject_averaged_TS.relevance_type.unique():

            stim_rel_data = subject_averaged_TS.query(f"stimulus_type == '{this_stim}' & relevance_type == '{this_rel}'")

            # Filter to specific regions during onset
            CS_onset = zscore(stim_rel_data.query("times >= 0 & times <= 1").Category_Selective.values)
            PFC_onset = zscore(stim_rel_data.query("times >= 0 & times <= 1").Prefrontal_Cortex.values)
            VIS_onset = zscore(stim_rel_data.query("times >= 0 & times <= 1").V1_V2.values)

            # Filter to specific regions during offset
            CS_offset = zscore(stim_rel_data.query("times > 1").Category_Selective.values)
            PFC_offset = zscore(stim_rel_data.query("times > 1").Prefrontal_Cortex.values)
            VIS_offset = zscore(stim_rel_data.query("times > 1").V1_V2.values)

            # Iterate over the barycenter methods in barycenter_method_dict
            for method_name, method_func in barycenter_method_dict.items():

                print(f"Computing {method_name} barycenter for {subject_ID}, {this_stim}, {this_rel}")

                # Compute the time-resolved barycenter for each region--region pair
                CS_PFC_onset_barycenter = method_func([CS_onset, PFC_onset])
                CS_VIS_onset_barycenter = method_func([CS_onset, VIS_onset])
                CS_PFC_offset_barycenter = method_func([CS_offset, PFC_offset])
                CS_VIS_offset_barycenter = method_func([CS_offset, VIS_offset])

                # Store (A) mean, (B) max, (C) time index of the max
                CS_PFC_onset_barycenter_mean = np.mean(CS_PFC_onset_barycenter)
                CS_PFC_onset_barycenter_max = np.max(CS_PFC_onset_barycenter)
                CS_PFC_onset_barycenter_max_time = np.argmax(CS_PFC_onset_barycenter)

                CS_VIS_onset_barycenter_mean = np.mean(CS_VIS_onset_barycenter)
                CS_VIS_onset_barycenter_max = np.max(CS_VIS_onset_barycenter)
                CS_VIS_onset_barycenter_max_time = np.argmax(CS_VIS_onset_barycenter)

                CS_PFC_offset_barycenter_mean = np.mean(CS_PFC_offset_barycenter)
                CS_PFC_offset_barycenter_max = np.max(CS_PFC_offset_barycenter)
                CS_PFC_offset_barycenter_max_time = np.argmax(CS_PFC_offset_barycenter)

                CS_VIS_offset_barycenter_mean = np.mean(CS_VIS_offset_barycenter)
                CS_VIS_offset_barycenter_max = np.max(CS_VIS_offset_barycenter)
                CS_VIS_offset_barycenter_max_time = np.argmax(CS_VIS_offset_barycenter)

                # Create dataframe
                stim_rel_df = pd.DataFrame({ "Subject": subject_ID,
                                            "Stimulus": this_stim,
                                            "Relevance": this_rel,
                                            "Presentation": ["Onset", "Onset", "Offset", "Offset"],
                                            "Region": ["CS_PFC", "CS_VIS", "CS_PFC", "CS_VIS"],
                                            "Method": [method_name, method_name, method_name, method_name],
                                            "Mean": [CS_PFC_onset_barycenter_mean, CS_VIS_onset_barycenter_mean, CS_PFC_offset_barycenter_mean, CS_VIS_offset_barycenter_mean],
                                            "Max": [CS_PFC_onset_barycenter_max, CS_VIS_onset_barycenter_max, CS_PFC_offset_barycenter_max, CS_VIS_offset_barycenter_max],
                                            "Max_Time": [CS_PFC_onset_barycenter_max_time, CS_VIS_onset_barycenter_max_time, CS_PFC_offset_barycenter_max_time, CS_VIS_offset_barycenter_max_time]
                                           })
                
                # Append to all_barycenter_res_list
                all_barycenter_res_list.append(stim_rel_df)
                
all_barycenter_res = pd.concat(all_barycenter_res_list)

Computing euclidean barycenter for sub-CB081, False, Irrelevant
Computing dtw barycenter for sub-CB081, False, Irrelevant
Computing sgddtw barycenter for sub-CB081, False, Irrelevant


/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/tslearn/barycenters/dba.py:735: ConvergenceWarning: DBA loss is increasing while it should not be. Stopping optimization.
  warnings.warn("DBA loss is increasing while it should not be. "


Computing euclidean barycenter for sub-CB081, False, Relevant non-target
Computing dtw barycenter for sub-CB081, False, Relevant non-target
Computing sgddtw barycenter for sub-CB081, False, Relevant non-target


/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/tslearn/barycenters/dba.py:735: ConvergenceWarning: DBA loss is increasing while it should not be. Stopping optimization.
  warnings.warn("DBA loss is increasing while it should not be. "


Computing euclidean barycenter for sub-CB081, False, Relevant target
Computing dtw barycenter for sub-CB081, False, Relevant target
Computing sgddtw barycenter for sub-CB081, False, Relevant target


...

/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/tslearn/barycenters/dba.py:735: ConvergenceWarning: DBA loss is increasing while it should not be. Stopping optimization.
  warnings.warn("DBA loss is increasing while it should not be. "


Computing euclidean barycenter for sub-CB081, object, Relevant non-target
Computing dtw barycenter for sub-CB081, object, Relevant non-target
Computing sgddtw barycenter for sub-CB081, object, Relevant non-target


/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/tslearn/barycenters/dba.py:735: ConvergenceWarning: DBA loss is increasing while it should not be. Stopping optimization.
  warnings.warn("DBA loss is increasing while it should not be. "


Computing euclidean barycenter for sub-CB081, object, Relevant target
Computing dtw barycenter for sub-CB081, object, Relevant target
Computing sgddtw barycenter for sub-CB081, object, Relevant target


/Users/abry4213/anaconda3/envs/annie_env/lib/python3.9/site-packages/tslearn/barycenters/dba.py:735: ConvergenceWarning: DBA loss is increasing while it should not be. Stopping optimization.
  warnings.warn("DBA loss is increasing while it should not be. "


In [4]:
                    stim_rel_df = pd.DataFrame({ "Subject": subject_ID,
                                                "Stimulus": this_stim,
                                                "Relevance": this_rel,
                                                "Presentation": ["Onset", "Onset", "Offset", "Offset"],
                                                "Region": ["CS_PFC", "CS_VIS", "CS_PFC", "CS_VIS"],
                                                "Method": [method_name, method_name, method_name, method_name],
                                                "Mean": [CS_PFC_onset_barycenter_mean, CS_VIS_onset_barycenter_mean, CS_PFC_offset_barycenter_mean, CS_VIS_offset_barycenter_mean],
                                                "Max": [CS_PFC_onset_barycenter_max, CS_VIS_onset_barycenter_max, CS_PFC_offset_barycenter_max, CS_VIS_offset_barycenter_max],
                                                "Max_Time": [CS_PFC_onset_barycenter_max_time, CS_VIS_onset_barycenter_max_time, CS_PFC_offset_barycenter_max_time, CS_VIS_offset_barycenter_max_time]
                                            })